CELL 1: SETUP AND IMPORTS

In [ ]:
# --- Core Physics & Math Library ---
import numpy as np  # Handles all vector states (spins), weight matrices, and linear algebra operations.

# --- Data Visualization Libraries ---
import matplotlib.pyplot as plt  # For plotting digit patterns and state trajectories.
import seaborn as sns  # For generating heatmaps and plots.

# --- Experiment Management ---
import pandas as pd  # Stores simulation results (accuracy, entropy, noise levels) for analysis.
from tqdm.notebook import tqdm  # Provides progress bars to track long-running simulations.

# --- Dataset Loader ---
import tensorflow as tf  # utilized for downloading the MNIST dataset.

# Configure plotting
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("paper", font_scale=1.4)

print("Computational environment initialized successfully.")

CELL 2: EXPERIMENT CONFIGURATION

In [ ]:
# Configuration dictionary centralizing all simulation parameters and physical constants.
CONFIG = {
    # System dimensionality (N), matching the 784 pixels of a 28x28 MNIST image.
    "NUM_NEURONS": 784,

    # List of digit classes (0-9) designated for memory pattern storage.
    "DIGITS_TO_STORE": list(range(10)),

    # Ratio of inhibitory neurons to be tested: 0.0 (symmetric) and 0.2 (asymmetric).
    "INHIBITORY_FRACTIONS_TO_TEST": [0.0, 0.2],

    # Noise range for robustness testing, simulating thermal fluctuations from 0% to 50%.
    "NOISE_LEVELS_EXP1": np.linspace(0.0, 0.5, 11),

    # Range of stored patterns (M) used to evaluate the network load parameter.
    "CAPACITY_TEST_PATTERNS_EXP3": list(range(1, 11)),

    # Number of independent trials per condition to ensure statistical significance.
    "NUM_RUNS_PER_CONDITION": 20,

    # Upper limit for asynchronous update iterations to ensure system relaxation.
    "MAX_RECALL_STEPS": 1500,

    # Length of subsequences compared during Sample Entropy (SampEn) calculation.
    "SAMPEN_M": 2,

    # Similarity threshold for SampEn, set to 20% of the energy signal's standard deviation.
    "SAMPEN_R_FRACTION": 0.2
}

print("Experimental configuration loaded successfully.")

CELL 3: CORE DEFINITIONS

In [ ]:
import numpy as np
from numba import jit

# ---------------------------------------------------------
# OPTIMIZED CORE: JIT-COMPILED DYNAMICS
# ---------------------------------------------------------

@jit(nopython=True)
def _fast_recall_loop(current_state, weights, max_steps, record_energy):
    """
    Executes the asynchronous update loop optimized with Numba for high-speed computation.
    Updates system energy incrementally (O(N)) to avoid redundant matrix operations.
    """
    N = len(current_state)
    energy_history = np.zeros(max_steps)

    # Initial system energy calculation: E = -0.5 * S^T * W * S.
    initial_field = weights.astype(np.float64) @ current_state.astype(np.float64)
    current_energy = -0.5 * np.dot(current_state.astype(np.float64), initial_field)

    for step in range(max_steps):
        # Stochastic selection of a single neuron index.
        idx = np.random.randint(0, N)

        # Calculation of the local field h_i acting on the selected neuron.
        local_field = 0.0
        for j in range(N):
            local_field += weights[idx, j] * current_state[j]

        # Application of the activation function: s_i = sign(h_i).
        old_spin = current_state[idx]
        new_spin = 1.0 if local_field >= 0 else -1.0

        # Update system state and energy only if a spin-flip occurs.
        if new_spin != old_spin:
            current_state[idx] = new_spin

            if record_energy:
                # Calculation of column sum to account for potential weight matrix asymmetry.
                col_sum = 0.0
                for k in range(N):
                    col_sum += weights[k, idx] * current_state[k]

                # Incremental energy update based on the spin-flip magnitude and total field influence.
                delta_s = new_spin - old_spin
                delta_term = delta_s * (local_field + col_sum)
                current_energy -= 0.5 * delta_term

        if record_energy:
            energy_history[step] = current_energy

    return current_state, energy_history

# ---------------------------------------------------------
# WRAPPER CLASS
# ---------------------------------------------------------

class HopfieldNetwork:
    """
    Main interface for network initialization, Hebbian training, and pattern retrieval.
    """
    def __init__(self, num_neurons, inhibitory_fraction=0.0):
        # Initialization of the weight matrix and identification of inhibitory neurons.
        self.N = num_neurons
        self.weights = np.zeros((num_neurons, num_neurons), dtype=np.float64)
        self.inhibitory_neurons = []

        if inhibitory_fraction > 0:
            num_inhibitory = int(self.N * inhibitory_fraction)
            self.inhibitory_neurons = np.random.choice(range(self.N), size=num_inhibitory, replace=False)

    def train(self, patterns):
        # Application of the Hebbian learning rule to store pattern prototypes.
        M = len(patterns)
        if M == 0: return

        # Computation of the weight matrix as the normalized sum of outer products.
        self.weights = np.sum([np.outer(p, p) for p in patterns], axis=0).astype(np.float64)
        self.weights /= M
        np.fill_diagonal(self.weights, 0) # Zeroing self-connections to prevent trivial attractors.

        # Enforcement of Dale's Principle by making selected neuron outputs purely inhibitory.
        if len(self.inhibitory_neurons) > 0:
            self.weights[self.inhibitory_neurons, :] *= -1

    def recall(self, initial_pattern, max_steps=100, record_energy=True):
        # Prepares initial state and invokes the optimized JIT recall loop.
        state = initial_pattern.astype(np.float64).copy()
        final_state, history = _fast_recall_loop(state, self.weights, int(max_steps), record_energy)
        return final_state, history

    def calculate_energy(self, state):
        # Standard matrix-based energy calculation for verification purposes.
        return -0.5 * np.dot(state.T, np.dot(self.weights, state))

# ---------------------------------------------------------
# ANALYSIS & VISUALIZATION UTILITIES
# ---------------------------------------------------------

def calculate_sampen(time_series, m=2, r_fraction=0.2):
    """
    Calculates Sample Entropy using Chebyshev distance to quantify energy trajectory complexity.
    Determines regularity by comparing matching template sequences of length m and m+1.
    """
    N = len(time_series)
    if N < m + 1: return 0
    std_dev = np.std(time_series)
    if std_dev == 0: return 0
    r = r_fraction * std_dev

    # Generation of template sequences for dimensionality comparison.
    templates_m = np.array([time_series[i : i + m] for i in range(N - m)])
    templates_m1 = np.array([time_series[i : i + m + 1] for i in range(N - m)])

    # Counting matches within tolerance r for sequences of length m.
    diff_m = np.abs(templates_m[:, None, :] - templates_m[None, :, :])
    matches_m = np.all(diff_m <= r, axis=-1)
    B = np.sum(matches_m) - len(templates_m) # Excludes self-matches.

    # Counting matches within tolerance r for sequences of length m+1.
    diff_m1 = np.abs(templates_m1[:, None, :] - templates_m1[None, :, :])
    matches_m1 = np.all(diff_m1 <= r, axis=-1)
    A = np.sum(matches_m1) - len(templates_m1)

    # Entropy calculation as the negative log of the conditional probability A/B.
    if A > 0 and B > 0:
        return -np.log(A / B)
    return 0

def calculate_overlap(state1, state2):
    # Measures normalized similarity (dot product) between two system states.
    return np.dot(state1, state2) / len(state1)

def plot_digit(digit_vector, ax, title=""):
    # Visualizes a flattened 784-element state vector as a 28x28 grayscale image.
    ax.imshow(digit_vector.reshape(28, 28), cmap='gray')
    ax.set_title(title)
    ax.axis('off')

print("Optimized Core Definitions ready (Numba Enabled).")

CELL 4: LOADING AND PREPROCESSING THE MNIST DATASET

In [ ]:
print("\n--- Loading and preparing the MNIST dataset ---")

# Retrieval of the MNIST dataset via TensorFlow's Keras API.
(x_train, y_train), (_, _) = tf.keras.datasets.mnist.load_data()

# Data preprocessing: flattening the 28x28 images into 784-element vectors
# and binarizing intensity values (Mapping: >127 to +1, otherwise -1)
# to align with the bipolar spin states of the Hopfield model.
x_train_binary = np.where(x_train.reshape((x_train.shape[0], -1)) > 127, 1, -1)

# Selection of specific digit classes for memory storage based on CONFIG parameters.
digits_to_store = CONFIG["DIGITS_TO_STORE"]

# Identification of the first occurrence index for each target digit in the dataset.
indices = [np.where(y_train == i)[0][0] for i in digits_to_store]

# Mapping of digit labels to their corresponding binarized pattern vectors for memory initialization.
all_patterns = {digit: x_train_binary[index] for digit, index in zip(digits_to_store, indices)}

print(f"{len(all_patterns)} unique digit prototypes are ready for the experiments.")

CELL 5: EXPERIMENT 1 & 2 - NOISE ROBUSTNESS AND DYNAMIC ANALYSIS

In [ ]:
def run_noise_robustness_experiment(config, patterns_to_store):
    """
    Executes a comprehensive experiment to evaluate network stability against stochastic noise.
    """
    all_results = []
    patterns = list(patterns_to_store.values())

    # Iteration over specified network architectures: symmetric and asymmetric.
    for frac in config["INHIBITORY_FRACTIONS_TO_TEST"]:
        print(f"\n--- Running Robustness Experiment: Inhibitory Fraction = {frac} ---")

        # Initialization and Hebbian training of the network for the current topology variant.
        net = HopfieldNetwork(config["NUM_NEURONS"], inhibitory_fraction=frac)
        net.train(patterns)

        # Systematic sweep of noise levels (bit-flip probabilities) to identify phase transition thresholds.
        for noise in tqdm(config["NOISE_LEVELS_EXP1"], desc=f"Noise Levels (Inhibition={frac})"):

            # Ensemble averaging loop to ensure statistical robustness and minimize stochastic bias.
            for _ in range(config["NUM_RUNS_PER_CONDITION"]):
                # Selection of a random target pattern from the stored memory set.
                original_pattern = patterns[np.random.randint(len(patterns))]

                # Generation of a corrupted initial state by flipping a fraction of bits equivalent to the noise level.
                initial_pattern = original_pattern.copy()
                num_to_flip = int(noise * config["NUM_NEURONS"])
                flip_indices = np.random.choice(range(config["NUM_NEURONS"]), size=num_to_flip, replace=False)
                initial_pattern[flip_indices] *= -1

                # Execution of the asynchronous recall process to reach an attractor state.
                final_state, energy_history = net.recall(initial_pattern, max_steps=config["MAX_RECALL_STEPS"])

                # Calculation of retrieval fidelity (final overlap) and dynamical trajectory complexity (SampEn).
                accuracy = calculate_overlap(final_state, original_pattern)
                entropy = calculate_sampen(energy_history, m=config["SAMPEN_M"], r_fraction=config["SAMPEN_R_FRACTION"])

                # Archiving trial-specific data into a structured record.
                all_results.append({
                    'noise': noise,
                    'accuracy': accuracy,
                    'entropy': entropy,
                    'inhibitory_fraction': frac
                })

    # Consolidation of experimental data into a Pandas DataFrame for statistical analysis.
    return pd.DataFrame(all_results)

# Selection of a four-digit subset (0, 1, 2, 8) to establish the initial load for the robustness analysis.
patterns_for_exp1 = {d: p for d, p in all_patterns.items() if d in [0, 1, 2, 8]}
df_exp1_results = run_noise_robustness_experiment(CONFIG, patterns_for_exp1)
print("\nRobustness Experiment completed.")

CELL 6: EXPERIMENT 3 - MEMORY CAPACITY

In [ ]:
def run_capacity_experiment(config, all_patterns):
    """
    Evaluates the storage capacity limits of the network by measuring retrieval fidelity
    as the number of stored patterns (memory load) increases.
    """
    all_results = []

    # Iteration over network topology variants: Standard (0.0) and Inhibitory (0.2).
    for frac in config["INHIBITORY_FRACTIONS_TO_TEST"]:
        print(f"\n--- Running Capacity Experiment: Inhibitory Fraction = {frac} ---")

        # Incremental sweep of the number of stored memory patterns from 1 to 10.
        for num_patterns in tqdm(config["CAPACITY_TEST_PATTERNS_EXP3"], desc=f"Number of Patterns (Inhibition={frac})"):

            # Retrieval of the subset of digit patterns for the current storage load.
            current_patterns_to_store = [all_patterns[d] for d in range(num_patterns)]

            # Re-initialization and training of the network for each specific pattern count.
            net = HopfieldNetwork(config["NUM_NEURONS"], inhibitory_fraction=frac)
            net.train(current_patterns_to_store)

            total_accuracy = 0
            # Evaluation of recall success for every individual stored pattern to calculate an average.
            for original_pattern in current_patterns_to_store:
                # Introduction of a fixed 10% noise level to probe the stability of the attractor basins.
                initial_pattern = original_pattern.copy()
                num_to_flip = int(0.1 * config["NUM_NEURONS"])
                flip_indices = np.random.choice(range(config["NUM_NEURONS"]), size=num_to_flip, replace=False)
                initial_pattern[flip_indices] *= -1

                # Execution of the recall process with energy recording disabled to optimize speed.
                final_state, _ = net.recall(initial_pattern, max_steps=config["MAX_RECALL_STEPS"], record_energy=False)
                total_accuracy += calculate_overlap(final_state, original_pattern)

            # Normalization of the cumulative accuracy by the number of patterns stored.
            average_accuracy = total_accuracy / num_patterns

            # Archiving of the mean retrieval fidelity for each load condition and architecture.
            all_results.append({
                'num_patterns': num_patterns,
                'avg_accuracy': average_accuracy,
                'inhibitory_fraction': frac
            })

    # Conversion of experimental metrics into a structured DataFrame for comparative analysis.
    return pd.DataFrame(all_results)

# Execution of the storage capacity experiment across all defined configurations.
df_exp3_results = run_capacity_experiment(CONFIG, all_patterns)
print("\nMemory Capacity Experiment completed.")

CELL 7: PLOTTING

In [ ]:
def generate_report_plots(df1, df3):
    """
    Generates report visualizations including phase space and centroid analysis.
    Filters anti-patterns to ensure physically consistent evaluation of recall dynamics.
    """
    print("\n--- Generating Individual Plots with Centroid Analysis ---")
    plt.style.use('seaborn-v0_8-whitegrid')

    # Mapping configuration labels to descriptive legend names for clarity.
    df1['Network Type'] = df1['inhibitory_fraction'].map({0.0: 'Standard', 0.2: 'Inhibitory (20%)'})
    df3['Network Type'] = df3['inhibitory_fraction'].map({0.0: 'Standard', 0.2: 'Inhibitory (20%)'})

    # --- Plot 1: Accuracy vs. Noise (Experiment 1) ---
    plt.figure(figsize=(10, 7))
    # Visualization of the paramagnetic transition as a function of stochastic noise.
    sns.lineplot(data=df1, x='noise', y='accuracy', hue='Network Type', errorbar='sd', palette='viridis', marker='o')
    plt.title('Experiment 1: Robustness to Noise', fontsize=16)
    plt.xlabel('Noise Level (Fraction of Flipped Bits)', fontsize=14)
    plt.ylabel('Recall Accuracy (Overlap)', fontsize=14)
    plt.ylim(-1.1, 1.1) # Range preserved to display initial distribution before filtering.
    plt.grid(True)
    plt.savefig('noise_robustness_plot.png', dpi=300, bbox_inches='tight')
    plt.show()

    # --- Plot 2: Phase Space and Centroid Analysis (Experiment 2) ---
    # Filtering: exclusion of negative overlaps (anti-patterns) to isolate correct retrieval dynamics.
    df1_filtered = df1[df1['accuracy'] > 0].copy()

    # Calculation of centroids representing mean accuracy and entropy for each architecture.
    centroids = df1_filtered.groupby('Network Type')[['accuracy', 'entropy']].mean().reset_index()

    plt.figure(figsize=(11, 8))
    # Distribution of filtered trials across the accuracy-entropy phase space.
    sns.scatterplot(
        data=df1_filtered,
        x='accuracy',
        y='entropy',
        hue='Network Type',
        style='Network Type',
        alpha=0.4,
        palette='viridis'
    )

    # Insertion of centroids as distinct 'X' markers for quantitative comparison.
    for i, row in centroids.iterrows():
        color = 'darkblue' if row['Network Type'] == 'Standard' else 'darkgreen'
        plt.scatter(
            row['accuracy'],
            row['entropy'],
            s=250,
            color=color,
            marker='X',
            edgecolor='white',
            linewidth=2,
            label=f"Centroid {row['Network Type']}"
        )
        # Annotation of numerical coordinates for data transparency.
        plt.text(
            row['accuracy'],
            row['entropy'] + 0.0001,
            f"({row['accuracy']:.3f}, {row['entropy']:.5f})",
            fontsize=10,
            horizontalalignment='center',
            color=color,
            weight='bold'
        )

    plt.title('Phase Space: Trajectory Complexity with Centroid Analysis', fontsize=16)
    plt.xlabel('Final Accuracy (Overlap > 0)', fontsize=14)
    plt.ylabel('Trajectory Complexity (Sample Entropy)', fontsize=14)
    plt.axvline(0.9, color='red', linestyle='--', alpha=0.5, label='Success Threshold')
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    plt.grid(True)
    plt.savefig('phase_space_complexity_centroids.png', dpi=300, bbox_inches='tight')
    plt.show()

    # --- Plot 3: Memory Capacity (Experiment 3) ---
    plt.figure(figsize=(10, 7))
    # Evaluation of catastrophic forgetting as a function of the storage load parameter alpha.
    sns.lineplot(data=df3, x='num_patterns', y='avg_accuracy', hue='Network Type', errorbar='sd', palette='viridis', marker='o')
    plt.title('Experiment 3: Memory Capacity', fontsize=16)
    plt.xlabel('Number of Stored Patterns', fontsize=14)
    plt.ylabel('Average Recall Accuracy', fontsize=14)
    plt.ylim(-0.1, 1.1)
    if "CAPACITY_TEST_PATTERNS_EXP3" in CONFIG:
        plt.xticks(CONFIG["CAPACITY_TEST_PATTERNS_EXP3"])
    plt.grid(True)
    plt.savefig('memory_capacity_plot.png', dpi=300, bbox_inches='tight')
    plt.show()

    # Tabular output for quantitative centroid verification in the report.
    print("\nCentroid Values (Quantitative Analysis):")
    print(centroids)

# Execution of visualization suite using collected experimental data.
generate_report_plots(df_exp1_results, df_exp3_results)

CELL 8: STATE SPACE TRAJECTORY

In [ ]:
# Function to visualize the system's trajectory within a 2D projection of the energy landscape.
# The state space is defined by the network's similarity (overlap) to two stored memory patterns.

def run_and_plot_trajectory(ax, inhibitory_fraction, patterns_to_use, noise_level=0.4):
    """
    Executes a single recall instance and plots the evolution of the state vector
    relative to two competing attractors in the state space.
    """
    # --- 1. Experimental Setup ---
    # Selection of visually similar memory prototypes ('3' and '8') to create a challenging landscape.
    pattern_A = patterns_to_use[3]
    pattern_B = patterns_to_use[8]

    # Initialization and training of the network variant (Standard or Inhibitory).
    net = HopfieldNetwork(CONFIG["NUM_NEURONS"], inhibitory_fraction=inhibitory_fraction)
    net.train([pattern_A, pattern_B])

    # Generation of a corrupted initial state from pattern '3' using a defined noise level.
    initial_state = pattern_A.copy()
    num_to_flip = int(noise_level * CONFIG["NUM_NEURONS"])
    flip_indices = np.random.choice(range(CONFIG["NUM_NEURONS"]), size=num_to_flip, replace=False)
    initial_state[flip_indices] *= -1

    # --- 2. Recall Dynamics and State Recording ---
    # Tracking of the system state throughout the asynchronous update process.
    state_history = [initial_state.copy()]
    current_state = initial_state.copy()
    for _ in range(CONFIG["MAX_RECALL_STEPS"]):
        # Stochastic neuron selection and state update based on the local field.
        neuron_to_update = np.random.randint(CONFIG["NUM_NEURONS"])
        local_field = np.dot(net.weights[neuron_to_update, :], current_state)
        new_state_i = 1 if local_field >= 0 else -1
        current_state[neuron_to_update] = new_state_i
        state_history.append(current_state.copy())

    # --- 3. Trajectory Projection ---
    # Translation of high-dimensional neural states into 2D coordinates via overlap calculation.
    overlap_A_history = [calculate_overlap(s, pattern_A) for s in state_history]
    overlap_B_history = [calculate_overlap(s, pattern_B) for s in state_history]

    # --- 4. Plotting and Visualization ---
    # Rendering of the temporal path using a viridis color gradient to indicate time progression.
    num_steps = len(overlap_A_history)
    colors = plt.cm.viridis(np.linspace(0, 1, num_steps))

    # Plotting the sequential state transitions in the projected overlap space.
    for i in range(num_steps - 1):
        ax.plot(overlap_A_history[i:i+2], overlap_B_history[i:i+2], color=colors[i], lw=1.5)

    # Labeling of start and end points to differentiate between noise and convergence.
    ax.scatter(overlap_A_history[0], overlap_B_history[0], color='blue', s=100, zorder=10, label='Start (Noisy \'3\')')
    ax.scatter(overlap_A_history[-1], overlap_B_history[-1], color='red', s=100, zorder=10, marker='X', label='End (Final State)')

    # Visualization of the target memory coordinates within the phase portrait.
    ax.plot(1, calculate_overlap(pattern_A, pattern_B), 'go', markersize=10, label='Perfect \'3\' Memory')
    ax.plot(calculate_overlap(pattern_B, pattern_A), 1, 'yo', markersize=10, label='Perfect \'8\' Memory')

    # Formatting of axes and grid for formal report presentation.
    ax.set_title(f'Network Type: {"Inhibitory" if inhibitory_fraction > 0 else "Standard"}', fontsize=16)
    ax.set_xlabel("Overlap with Digit '3'", fontsize=12)
    ax.set_ylabel("Overlap with Digit '8'", fontsize=12)
    ax.grid(True, linestyle='--', alpha=0.6)
    ax.legend()
    ax.set_xlim(0, 1.1)
    ax.set_ylim(0, 1.1)
    ax.set_aspect('equal', adjustable='box')


# --- Main Visualization Routine ---
# Side-by-side comparison of standard symmetric dynamics versus inhibitory non-equilibrium trajectories.
fig, axes = plt.subplots(1, 2, figsize=(18, 8))
fig.suptitle('State Space Trajectory During Memory Recall', fontsize=20)

# Execution for the standard Hebbian architecture.
run_and_plot_trajectory(axes[0], inhibitory_fraction=0.0, patterns_to_use=all_patterns)

# Execution for the biological inhibitory architecture.
run_and_plot_trajectory(axes[1], inhibitory_fraction=0.2, patterns_to_use=all_patterns)

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

CELL 9: THE ANATOMY OF MEMORY (WEIGHT MATRIX)

In [ ]:
# Function to train the network and visualize the resulting synaptic connectivity structure.
# This analysis provides a structural baseline to compare symmetric and asymmetric architectures.

def run_and_plot_weight_matrix(ax, inhibitory_fraction, patterns_to_use, title):
    """
    Executes network training and generates a heatmap of the synaptic weight matrix (W).
    - ax: Matplotlib axis for rendering.
    - inhibitory_fraction: Fraction of neurons designated as inhibitory (Dale's Principle).
    - patterns_to_use: Dictionary containing training prototypes.
    - title: Subplot identification label.
    """
    # --- 1. Network Initialization and Training ---
    # Selection of a five-digit pattern subset (0-4) for memory initialization.
    patterns = [patterns_to_use[d] for d in [0, 1, 2, 3, 4]]

    # Instantiation of the Hopfield network with the specified inhibitory ratio.
    net = HopfieldNetwork(CONFIG["NUM_NEURONS"], inhibitory_fraction=inhibitory_fraction)

    # Application of the Hebbian learning rule to populate the weight matrix.
    net.train(patterns)

    # --- 2. Structural Visualization ---
    # Generation of a heatmap using a perceptually uniform colormap to represent weight magnitudes.
    # This reveals the physical distribution of stored memory correlations.
    sns.heatmap(net.weights, ax=ax, cmap='viridis', cbar=True)

    ax.set_title(title, fontsize=16)

    # Suppression of individual neuron ticks (784 total) to maintain visual clarity.
    ax.set_xlabel("Neuron j")
    ax.set_ylabel("Neuron i")
    ax.set_xticks([])
    ax.set_yticks([])


# --- Main Visualization Routine ---
# Comparative display of standard symmetric weights versus asymmetric inhibitory structures.
fig, axes = plt.subplots(1, 2, figsize=(18, 8))
fig.suptitle('Visualization of the Synaptic Weight Matrix (W)', fontsize=20)



# Execution for the Standard Network, illustrating perfect symmetry across the diagonal (w_ij = w_ji).
run_and_plot_weight_matrix(axes[0],
                           inhibitory_fraction=0.0,
                           patterns_to_use=all_patterns,
                           title="Standard Network (Symmetric)")

# Execution for the Inhibitory Network, highlighting the "striped" pattern caused by unidirectional inhibitory columns.
run_and_plot_weight_matrix(axes[1],
                           inhibitory_fraction=0.2,
                           patterns_to_use=all_patterns,
                           title="Inhibitory Network (Asymmetric)")

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

CELL 10: ANIMATED RECALL

In [ ]:
# Imports required for the generation and browser-based rendering of dynamic neural simulations.
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def run_and_generate_animation(inhibitory_fraction, patterns_to_use, digit_to_test=8, noise_level=0.2, animation_steps=800):
    """
    Simulates the recall process and compiles the state evolution into a GIF animation for qualitative analysis.
    """
    print(f"\n--- Generating Animation (Network: {'Inhibitory' if inhibitory_fraction > 0 else 'Standard'}) ---")

    # Initialization of the network architecture and training using selected MNIST prototypes.
    patterns = [patterns_to_use[d] for d in [0, 1, 3, 5, 8]]
    net = HopfieldNetwork(CONFIG["NUM_NEURONS"], inhibitory_fraction=inhibitory_fraction)
    net.train(patterns)

    original_pattern = patterns_to_use[digit_to_test]

    # Application of bit-flip noise to the selected target pattern to test retrieval dynamics.
    noisy_pattern = original_pattern.copy()
    np.random.seed(43) # Fixed seed ensures consistency in initial state corruption across different runs.
    num_to_flip = int(noise_level * CONFIG["NUM_NEURONS"])
    flip_indices = np.random.choice(range(CONFIG["NUM_NEURONS"]), size=num_to_flip, replace=False)
    noisy_pattern[flip_indices] *= -1

    # Incremental recording of high-dimensional neural states during the asynchronous relaxation process.
    state_history = [noisy_pattern.copy()]
    current_state = noisy_pattern.copy()
    for _ in range(animation_steps):
        # Stochastic neuron selection and deterministic state update based on the local field sign.
        neuron_to_update = np.random.randint(CONFIG["NUM_NEURONS"])
        local_field = np.dot(net.weights[neuron_to_update, :], current_state)
        new_state_i = 1 if local_field >= 0 else -1
        current_state[neuron_to_update] = new_state_i

        # Sampling the system state at fixed intervals to optimize animation playback and generation speed.
        if _ % 5 == 0:
             state_history.append(current_state.copy())

    # Implementation of the visual rendering logic for the animation sequence.
    fig, ax = plt.subplots(figsize=(5, 5))
    plt.close() # Suppression of static figure output to prevent redundant rendering.

    def animate(i):
        # Clears and redraws the 28x28 neural grid for the current time step i.
        ax.clear()
        plot_digit(state_history[i], ax, title=f"Recall Step: {i*5}")

    # Aggregation of the recorded history into a temporal sequence for dynamic observation.
    anim = FuncAnimation(fig, animate, frames=len(state_history), interval=50, blit=False)

    # Conversion of the animation into HTML5 format for visual comparison of relaxation regimes.
    return HTML(anim.to_html5_video())

# --- Sequential execution of the animation routine for both network variants ---
# Generation of the dynamic retrieval sequence for the standard Hebbian architecture.
animation_standard = run_and_generate_animation(0.0, all_patterns)
print("Standard Network Animation Ready.")

# Generation of the dynamic retrieval sequence for the inhibitory architecture.
animation_inhibitory = run_and_generate_animation(0.2, all_patterns)
print("Inhibitory Network Animation Ready.")

# Display of the comparative animations within the Jupyter environment.
print("\n--- Recall Animation for Standard Network ---")
display(animation_standard)

print("\n--- Recall Animation for Inhibitory Network ---")
display(animation_inhibitory)

CELL 11: DEMONSTRATION OF THE FULL RECALL PROCESS

In [ ]:
# Parameters for memory retrieval demonstration.
DIGIT_TO_SHOW = 5  # Specific MNIST prototype selected for visual reconstruction.
NOISE_LEVEL_DEMO = 0.35  # Fraction of bits stochastically flipped to test recall robustness.
# A set of five distinct digits is used for training to ensure a non-trivial energy landscape.
PATTERNS_TO_TRAIN_ON = [0, 1, 3, 5, 8]

print(f"Creating full recall visualization for digit '{DIGIT_TO_SHOW}' with {int(NOISE_LEVEL_DEMO*100)}% noise.")

# --- 1. Data and Network Preparation ---
# Retrieval of the uncorrupted binarized prototype from the dataset.
original_pattern_demo = all_patterns[DIGIT_TO_SHOW]

# Corrupted input generation via state-flipping of 35% of randomly selected neurons.
noisy_pattern_demo = original_pattern_demo.copy()
num_to_flip_demo = int(NOISE_LEVEL_DEMO * CONFIG["NUM_NEURONS"])
np.random.seed(101) # Seed is fixed to ensure a reproducible noise distribution.
flip_indices_demo = np.random.choice(range(CONFIG["NUM_NEURONS"]), size=num_to_flip_demo, replace=False)
noisy_pattern_demo[flip_indices_demo] *= -1

# --- 2. Recall Process Execution ---
# Initialization and Hebbian training of a symmetric Hopfield network with 784 neurons.
patterns_for_training = [all_patterns[d] for d in PATTERNS_TO_TRAIN_ON]
net_demo = HopfieldNetwork(CONFIG["NUM_NEURONS"], inhibitory_fraction=0.0)
net_demo.train(patterns_for_training)

# Execution of the asynchronous update rule to drive the noisy input toward a stored attractor.
# Energy recording is disabled to maximize computational throughput during demonstration.
recalled_state_demo, _ = net_demo.recall(noisy_pattern_demo, max_steps=CONFIG["MAX_RECALL_STEPS"], record_energy=False)
final_overlap_demo = calculate_overlap(recalled_state_demo, original_pattern_demo)
print(f"Final overlap with original pattern: {final_overlap_demo:.3f}")

# --- 3. Visualization Generation ---
# Comparative layout illustrating the original prototype, corrupted input, and final reconstructed state.
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
fig.suptitle('Visual Demonstration of Associative Memory Recall', fontsize=16)

# Sequential plotting of patterns to illustrate the denoising and pattern completion capabilities.
plot_digit(original_pattern_demo, axes[0], title=f"Original Pattern ('{DIGIT_TO_SHOW}')")
plot_digit(noisy_pattern_demo, axes[1], title=f"Corrupted Input ({int(NOISE_LEVEL_DEMO*100)}% Noise)")
plot_digit(recalled_state_demo, axes[2], title="Reconstructed Pattern")

plt.tight_layout(rect=[0, 0, 1, 0.93])
plt.show()